In [1]:
import re
import pandas as pd
import numpy as np
import pymysql
import hgtk

"""
* 로마자 자동화V4.8 *

두레이 프로젝트링크 : https://redtable.dooray.com/project/important/3214559034218019477
구글 스프레드시트 : https://docs.google.com/spreadsheets/d/1mM7m7MP6IFZ5vwNUSwwJfFjd4a5uIYJ1KqFUGhlHWHg/edit#gid=0

업데이트일시 : 2023.04.19
업데이트내용 : 
    1) 표준발음법 코드 삭제
    2) "[붙임] 첫소리 'ㄴ'이 'ㅀ, ㄾ' 뒤에 연결되는 경우에도 이에 준한다." 삭제하기
    3) "다만, 다음과 같은 단어들은 'ㄹ'을 [ㄴ]으로 발음한다."  삭제하기
    4) "제3장 제1항 2. ‘ㄴ, ㄹ’이 덧나는 경우" 적용하기
        예시) 학여울(Hangnyeoul), 알약(Allyak)
        
담당자 : Hob
"""

"""
개발DB dictionary에 있는 테이블을 사전으로 활용하기
"""
db_host = 'db-7ma06.pub-cdb.ntruss.com'
db_user = 'redtable_openapi'
db_password = 'fpemxpdlqmf5491!@#'
db_port = 3306
db_name = 'dictionary'

conn = pymysql.connect(host=db_host, user=db_user, password=db_password, port=db_port, db=db_name, charset='utf8')
cursor = conn.cursor()

"""
#1 rome_dic_idiom : 로마자 외래어관용어 사전
#2 rome_rules : 로마자 관련 규칙적용표
"""

sql = "select * from dictionary.rome_dic_idiom"
cursor.execute(sql)
rome_dic_idiom = cursor.fetchall()
rome_dic_idiom = pd.DataFrame(rome_dic_idiom, columns=["id", "priority", "kr_word", "rome_word", "kr_word_len"])

except_dic = [
        '밟다','밟소','밟지','밟는','밟게','밟고','넓죽하다','넓둥글','넓죽한','넓적','넓적다리','넓다','맏형',
        '의견란','임진란','생산량','결단력','공권력','동원령','상견례','횡단로','이원론','입원료','구근류'
]
except_rom_dic = [
    'babtta','babso','babjji','bamneun','babkke','babkko','neopjjukada','neopttunggeul','neopjjukan','neopjjeok','neopjjeokddari','neoldda','mathyeong',
    'uigyeonnan','imjinnan','saengsannyang','gyeolttannyeok','gongkkwonnyeok','dongwonnyeong','sanggyeonnye','hoengdanno','iwonnon','ibwonnyo','gugeunnyu'
]

except_df = pd.DataFrame(data = list(zip(except_dic, except_rom_dic)), columns = ["kr_word", "rome_word"])
rome_dic_idiom = pd.concat([rome_dic_idiom, except_df], ignore_index=True)
arr = rome_dic_idiom.to_numpy()

sql = "select * from dictionary.rome_rules"
cursor.execute(sql)
rome_rules = cursor.fetchall()
rome = pd.DataFrame(rome_rules, columns=["id", "rome_rules_hangeul", "초성", "모음앞종성", "종성"])
rome_dic = rome['rome_rules_hangeul'].values

cursor.close()
conn.close()

def romanize(text):
    """
    오로지 한글만 들어오는 것 가능
    띄어쓰기 없는 단어
    결과 : 로마자(str)
    """
    result = ""
    for i in range(len(text)):
        letter = hgtk.letter.decompose(text[i])
        if i == 0:
            if letter[0] in rome_dic:
                result = "".join([result, rome['초성'][np.where(letter[0]==rome_dic)[0][0]]])
            if letter[1] in rome_dic:
                result = "".join([result, rome['초성'][np.where(letter[1]==rome_dic)[0][0]]])
            if i == (len(text)-1) and letter[2] in rome_dic:
                result = "".join([result, rome['종성'][np.where(letter[2]==rome_dic)[0][0]]])
            if i != (len(text)-1):
                letter2 = hgtk.letter.decompose(text[i+1])
                if letter[2] == 'ㄷ' and letter2[0] == 'ㅎ':
                    result = "".join([result, 'ch'])
                elif letter[2] == 'ㄹ' and letter2[0] == 'ㄹ':
                    result = "".join([result, 'l'])
                elif letter[2] == 'ㄺ' and letter2[0] == 'ㄱ':
                    result = "".join([result, 'l'])
                elif letter[2] == 'ㅎ' and (letter2[0] == 'ㄱ' or letter2[0] == 'ㄷ' or letter2[0] == 'ㅈ'):
                    result = "".join([result, '%'])
                elif letter[2] == 'ㄶ' and (letter2[0] == 'ㄱ' or letter2[0] == 'ㄷ' or letter2[0] == 'ㅈ'):
                    result = "".join([result, 'n'])
                elif letter[2] == 'ㅀ' and (letter2[0] == 'ㄱ' or letter2[0] == 'ㄷ' or letter2[0] == 'ㅈ'):
                    result = "".join([result, 'l'])
                elif letter[2] == 'ㄴ' and letter2[0] == 'ㄹ':
                    result = "".join([result, 'l'])
                elif (letter[2] == 'ㄹ' or letter[2] == 'ㅀ' or letter[2] == 'ㄾ') and letter2[0] == 'ㄴ':
                    result = "".join([result, 'l'])
                elif (letter[2] == 'ㄺ' or letter[2] == 'ㄼ' or letter[2] == 'ㅈ' or letter[2] == 'ㄵ') and letter2[0] == 'ㅎ':
                    result = "".join([result, '%'])
                elif letter[2] == 'ㄱ' and letter2[0] == 'ㅎ':
                    result = "".join([result, 'k'])
                elif letter[2] == 'ㅎ' and letter2[0] == 'ㅅ':
                    result = "".join([result, '%'])
                elif (letter[2] == 'ㅎ' or letter[2] == 'ㄶ') and letter2[0] == 'ㄴ':
                    result = "".join([result, 'n'])
                elif letter[2] == 'ㅀ' and letter2[0] == 'ㄴ':
                    result = "".join([result, 'l'])
                    
                    
                elif letter[2] == 'ㄹ' and letter2[0] == 'ㅇ':
                    result = "".join([result, 'l'])
                elif letter[2] == 'ㄱ' and letter2[0] == 'ㅇ':
                    result = "".join([result, 'ng'])
                    
                    
                elif letter[2] == 'ㅎ' and letter2[0] == 'ㅇ':
                    result = "".join([result, '%'])
                elif letter[2] == 'ㄶ' and letter2[0] == 'ㅇ':
                    result = "".join([result, 'n'])
                elif letter[2] == 'ㅀ' and letter2[0] == 'ㅇ':
                    result = "".join([result, 'r'])
                elif letter2[0] == 'ㅇ' and letter[2] in rome_dic:
                    if letter[2] == 'ㄷ' and letter2[1] == 'ㅣ':
                        result = "".join([result, 'j'])
                    elif letter[2] == 'ㅌ' and letter2[1] == 'ㅣ':
                        result = "".join([result, 'ch'])
                    elif letter[2] == 'ㅅ' and letter2[1] == 'ㅣ' and letter2[2] in rome_dic:
                        result = "".join([result, 'n'])
                    else:
                        result = "".join([result, rome['모음앞종성'][np.where(letter[2]==rome_dic)[0][0]]])
                elif letter2[0] != 'ㅇ' and letter[2] in rome_dic:
                    result = "".join([result, rome['종성'][np.where(letter[2]==rome_dic)[0][0]]])

        else:
            letter3 = hgtk.letter.decompose(text[i-1])
            if letter[0] in rome_dic:
                if letter[0] == 'ㅎ' and (letter3[2] == 'ㄱ' or letter3[2] == 'ㄷ' or letter3[2] == 'ㅂ'):
                    result = "".join([result, 'h'])
                elif letter[0] == 'ㄹ' and (letter3[2] == 'ㄹ' or letter3[2] == 'ㄴ'):
                    result = "".join([result, 'l'])
                elif letter[0] == 'ㄴ' and (letter3[2] == 'ㄹ' or letter3[2] == 'ㅀ' or letter3[2] == 'ㄾ'):
                    result = "".join([result, 'l'])
                elif letter[0] == 'ㄹ' and (letter3[2] == 'ㅁ' or letter3[2] == 'ㅇ' or letter3[2] == 'ㄱ' or letter3[2] == 'ㅂ'):
                    result = "".join([result, 'n'])
                elif letter[0] == 'ㄴ' and (letter3[2] == 'ㅎ' or letter3[2] == 'ㄶ'):
                    result = "".join([result, 'n'])
                elif letter[0] == 'ㄱ' and letter3[2] == 'ㄺ':
                    result = "".join([result, 'kk'])
                elif letter[0] == 'ㄱ' and (letter3[2] == 'ㅎ' or letter3[2] == 'ㄶ' or letter3[2] == 'ㅀ'):
                    result = "".join([result, 'k'])
                elif letter[0] == 'ㅎ' and letter3[2] == 'ㄺ':
                    result = "".join([result, 'k'])
                elif letter[0] == 'ㅈ' and (letter3[2] == 'ㅎ' or letter3[2] == 'ㄶ' or letter3[2] == 'ㅀ'):
                    result = "".join([result, 'ch'])
                elif letter[0] == 'ㅎ' and (letter3[2] == 'ㅈ' or letter3[2] == 'ㄵ'):
                    result = "".join([result, 'ch'])
                elif letter[0] == 'ㄷ' and (letter3[2] == 'ㅎ' or letter3[2] == 'ㄶ' or letter3[2] == 'ㅀ'):
                    result = "".join([result, 't'])
                elif letter[0] == 'ㅎ' and letter3[2] == 'ㄼ':
                    result = "".join([result, 'p'])
                elif letter[0] == 'ㅅ' and letter3[2] == 'ㅎ':
                    result = "".join([result, 'ss'])
                
                
                elif letter[0] == 'ㅇ' and letter3[2] == 'ㄹ':
                    result = "".join([result, 'l'])
                elif letter[0] == 'ㅇ' and letter3[2] == 'ㄱ':
                    result = "".join([result, 'n'])
                
                
                elif letter[0] == 'ㅇ' and letter[1] == 'l' and letter3[2] == 'ㅅ':
                    result = "".join([result, 'n'])
                else:
                    result = "".join([result, rome['초성'][np.where(letter[0]==rome_dic)[0][0]]])

            if letter[1] in rome_dic:
                result = "".join([result, rome['초성'][np.where(letter[1]==rome_dic)[0][0]]])
            if i == (len(text)-1) and letter[2] in rome_dic:
                result = "".join([result, rome['종성'][np.where(letter[2]==rome_dic)[0][0]]])
            if i != (len(text)-1):
                letter2 = hgtk.letter.decompose(text[i+1])
                if letter[2] == 'ㄷ' and letter2[0] == 'ㅎ':
                    result = "".join([result, 'ch'])
                elif letter[2] == 'ㄹ' and letter2[0] == 'ㄹ':
                    result = "".join([result, 'l'])
                elif letter[2] == 'ㄺ' and letter2[0] == 'ㄱ':
                    result = "".join([result, 'l'])
                elif letter[2] == 'ㅎ' and (letter2[0] == 'ㄱ' or letter2[0] == 'ㄷ' or letter2[0] == 'ㅈ'):
                    result = "".join([result, '%'])
                elif letter[2] == 'ㄶ' and (letter2[0] == 'ㄱ' or letter2[0] == 'ㄷ' or letter2[0] == 'ㅈ'):
                    result = "".join([result, 'n'])
                elif letter[2] == 'ㅀ' and (letter2[0] == 'ㄱ' or letter2[0] == 'ㄷ' or letter2[0] == 'ㅈ'):
                    result = "".join([result, 'l'])
                elif letter[2] == 'ㄴ' and letter2[0] == 'ㄹ':
                    result = "".join([result, 'l'])
                elif (letter[2] == 'ㄹ' or letter[2] == 'ㅀ' or letter[2] == 'ㄾ') and letter2[0] == 'ㄴ':
                    result = "".join([result, 'l'])
                elif (letter[2] == 'ㄺ' or letter[2] == 'ㅂ' or letter[2] == 'ㄼ' or letter[2] == 'ㅈ' or letter[2] == 'ㄵ') and letter2[0] == 'ㅎ':
                    result = "".join([result, '%'])
                elif letter[2] == 'ㄱ' and letter2[0] == 'ㅎ':
                    result = "".join([result, 'k'])
                elif letter[2] == 'ㅎ' and letter2[0] == 'ㅅ':
                    result = "".join([result, '%'])
                elif (letter[2] == 'ㅎ' or letter[2] == 'ㄶ') and letter2[0] == 'ㄴ':
                    result = "".join([result, 'n'])
                elif letter[2] == 'ㅀ' and letter2[0] == 'ㄴ':
                    result = "".join([result, 'l'])
                elif letter[2] == 'ㅎ' and letter2[0] == 'ㅇ':
                    result = "".join([result, '%'])
                elif letter[2] == 'ㄶ' and letter2[0] == 'ㅇ':
                    result = "".join([result, 'n'])
                elif letter[2] == 'ㅀ' and letter2[0] == 'ㅇ':
                    result = "".join([result, 'r'])
                    
                    
                elif letter[2] == 'ㄹ' and letter2[0] == 'ㅇ':
                    result = "".join([result, 'l'])
                elif letter[2] == 'ㄱ' and letter2[0] == 'ㅇ':
                    result = "".join([result, 'ng'])    
                    
                    
                elif letter2[0] == 'ㅇ' and letter[2] in rome_dic:
                    if letter[2] == 'ㄷ' and letter2[1] == 'ㅣ':
                        result = "".join([result, 'j'])
                    elif letter[2] == 'ㅌ' and letter2[1] == 'ㅣ':
                        result = "".join([result, 'ch'])
                    elif letter[2] == 'ㅅ' and letter2[1] == 'ㅣ' and letter2[2] in rome_dic:
                        result = "".join([result, 'n'])
                    else:
                        result = "".join([result, rome['모음앞종성'][np.where(letter[2]==rome_dic)[0][0]]])
                elif letter2[0] != 'ㅇ' and letter[2] in rome_dic:
                    result = "".join([result, rome['종성'][np.where(letter[2]==rome_dic)[0][0]]])

    result = result.replace(' ','').replace('%','')
    result = romanize2(result)
    return result

def romanize2(text):
    """
    추가적인 발음 규정 적용
    비음화, ㄴ 소리 첨가, 사이시옷
    """
    #비음화
    if 'kn' in text:
        text = text.replace('kn','ngn')
    elif 'km' in text:
        text = text.replace('km','ngm')
    elif 'tn' in text:
        text = text.replace('tn','nn')
    elif 'tm' in text:
        text = text.replace('tm','nm')
    elif 'pn' in text:
        text = text.replace('pn','mn')
    elif 'pm' in text:
        text = text.replace('pm','mm')
    
    return text

def foreign_dic(text):
    for i in arr:
        if len(i[2]) <= len(text):
            if i[2] in text:
                text = text.replace(i[2], ' '+i[3]+' ')
                continue
    return text

def isHangul(text):
    """
    text가 한글(자음,모음,단모음) 여부를 확인
    결과값은 True / False 로 나옴
    입    력 : ㄱa1
    결    과 : True
    """
    try:
        hanCount = len(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', text))
        return hanCount > 0
    except:
        return False
    
def isEng(text):
    try:
        if text.islower() != text.isupper():
            return True
        else:
            return False
    except:
        return False

def remove_blank(text):
    """
    앞뒤 공백 제거
    특수문자 앞뒤 공백제거
    """
    if text[0] == ' ':
        text = text[1:]
    if text[-1] == ' ':
        text = text[:-1]
    
    text = text.replace('   ',' ').replace('  ',' ')
            
    lis = ['(',')','+','-','_','=','&','#','@','!','?','/','\\','<','>','[',']','{','}',':',';','%','^','*',',']
    arr2 = np.array(lis)
    for i in arr2:
        if i in text:
            text = text.replace(i+' ',i).replace(' '+i,i)
            continue
            
    return text
    
def auto_romanize(input_text):
    hangul = re.compile('[가-힣]+')

    if type(input_text) != str:
        final_result = input_text
        return final_result
    
    if type(input_text) == str:
        if isHangul(input_text) == False and isEng(input_text) == False:
            final_result = input_text
            return final_result
    
    # 외래어 사전 적용 
    input_text = foreign_dic(input_text)
    
    while len(hangul.findall(input_text)) > 0:
        han = hangul.findall(input_text)
        for i in range(len(han)):
            han_re = romanize(han[i])
            input_text = input_text.replace(han[i],han_re)
            continue
    
    final_result = remove_blank(input_text)
    return final_result

In [2]:
auto_romanize("학여울")

'hangnyeoul'

In [3]:
auto_romanize("알약")

'allyak'

In [3]:
auto_romanize("에덴밸리카페테리아2")

'eden vally cafe teria 2'

In [2]:
auto_romanize("디비에이")

'dibiei'